In [30]:
# Transformers installation
! pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import numpy as np
import evaluate

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import RobertaForSequenceClassification
from datasets import load_dataset

In [32]:
dataset = load_dataset("rotten_tomatoes")

  0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=120)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--cardiffnlp--twitter-roberta-base-emotion/snapshots/dff452c4f42c15a25bd51aff1f1ca5d15ec08c23/config.json
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-emotion",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "joy",
    "1": "optimism",
    "2": "anger",
    "3": "sadness"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 2,
    "joy": 0,
    "optimism": 1,
    "sadness": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_

  0%|          | 0/2 [00:00<?, ?ba/s]

In [34]:
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--cardiffnlp--twitter-roberta-base-emotion/snapshots/dff452c4f42c15a25bd51aff1f1ca5d15ec08c23/config.json
Model config RobertaConfig {
  "_name_or_path": "tweeteval/roberta-base-rt-emotion/",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "joy",
    "1": "optimism",
    "2": "anger",
    "3": "sadness"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 2,
    "joy": 0,
    "optimism": 1,
    "sadness": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type":

In [35]:
training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [36]:
metric = evaluate.load("accuracy")

In [37]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [38]:
training_args = TrainingArguments(
    num_train_epochs=10,
    output_dir="/content/drive/MyDrive/rotten-tomatoes/RoBERTa", 
    evaluation_strategy="epoch"
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [46]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

In [48]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8530
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10670
  Number of trainable parameters = 124648708


Epoch,Training Loss,Validation Loss,Accuracy
1,0.109300,0.842839,0.863039
2,0.105900,0.689597,0.858349
3,0.078400,0.911505,0.851782
4,0.087100,0.944715,0.854597
5,0.061500,0.909791,0.863977
6,0.040600,0.955989,0.862101
7,0.029800,1.106104,0.848030
8,0.020300,1.078737,0.855535
9,0.011900,1.032677,0.864916
10,0.012700,1.020007,0.868668


Saving model checkpoint to /content/drive/MyDrive/rotten-tomatoes/RoBERTa/checkpoint-500
Configuration saved in /content/drive/MyDrive/rotten-tomatoes/RoBERTa/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/rotten-tomatoes/RoBERTa/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/rotten-tomatoes/RoBERTa/checkpoint-1000
Configuration saved in /content/drive/MyDrive/rotten-tomatoes/RoBERTa/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/rotten-tomatoes/RoBERTa/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1066
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/rotten-tomatoes/RoBERTa/checkpoint-

TrainOutput(global_step=10670, training_loss=0.05779819669517939, metrics={'train_runtime': 2514.0171, 'train_samples_per_second': 33.93, 'train_steps_per_second': 4.244, 'total_flos': 5260260009888000.0, 'train_loss': 0.05779819669517939, 'epoch': 10.0})

In [49]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1066
  Batch size = 8


{'eval_loss': 1.020006537437439,
 'eval_accuracy': 0.8686679174484052,
 'eval_runtime': 7.4432,
 'eval_samples_per_second': 143.218,
 'eval_steps_per_second': 18.003,
 'epoch': 10.0}

In [51]:
trainer.predict(tokenized_datasets['test'])

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


PredictionOutput(predictions=array([[ -2.4566562,   6.365389 , -12.513198 , -12.378356 ],
       [ -2.5512314,   6.3088555, -12.106256 , -11.968406 ],
       [  5.849636 ,  -2.4208002, -11.913176 , -11.758999 ],
       ...,
       [  5.821492 ,  -2.461056 , -11.733558 , -11.572339 ],
       [  5.813234 ,  -2.4727283, -11.684027 , -11.519427 ],
       [  5.8256617,  -2.4553838, -11.759026 , -11.598444 ]],
      dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 1.0828759670257568, 'test_accuracy': 0.8602251407129456, 'test_runtime': 9.4514, 'test_samples_per_second': 112.788, 'test_steps_per_second': 14.178})